# Capstone Project

### IBM Data Science Professional Certificate capstone project

In [1]:
import pandas as pd
import numpy as np
print('Hello Capstone Project Course!')

Hello Capstone Project Course!


## Assignment 1: Segmenting and Clustering Neighborhoods in Toronto

Week 3 assignment. 

### Step 1. Create the dataframe

In [2]:
!pip install beautifulsoup4

In [3]:
from bs4 import BeautifulSoup as bs
import requests

In [4]:
#Request dataframe from webpage

request=requests.get('https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M')
soup=bs(request.content, 'lxml')
table=soup.find_all('table')[0]
df_toronto=pd.read_html(str(table))[0]

df_toronto.head(10)

,Postal Code,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,"Regent Park, Harbourfront"
5,M6A,North York,"Lawrence Manor, Lawrence Heights"
6,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
7,M8A,Not assigned,Not assigned
8,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
9,M1B,Scarborough,"Malvern, Rouge"


In [5]:
df_toronto.shape

(180, 3)

In [6]:
#Check how many rows has "Not assigned" borough

df_toronto['Borough'].value_counts()

Not assigned        77
North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East York            5
East Toronto         5
York                 5
Mississauga          1
Name: Borough, dtype: int64

In [7]:
#Drop rows with "Not assigned" borough
df_toronto=df_toronto.drop(df_toronto[df_toronto['Borough']=='Not assigned'].index, axis=0)

#reset index
df_toronto.reset_index(drop=True, inplace=True)

#clean column name
df_toronto.rename(columns={'Neighbourhood':'Neighborhood', 'Postal Code':'PostalCode'}, inplace=True)

#check the dataframe
df_toronto.head(10)

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village"
6,M1B,Scarborough,"Malvern, Rouge"
7,M3B,North York,Don Mills
8,M4B,East York,"Parkview Hill, Woodbine Gardens"
9,M5B,Downtown Toronto,"Garden District, Ryerson"


In [8]:
print('Number of rows should be', 180-77, 'and new dataframe number of rows is', df_toronto.shape[0])

Number of rows should be 103 and new dataframe number of rows is 103


In [9]:
#Assign neighborhood name with borough name if Neighborhood=Not assigned

#1. See how many neighborhoods are not named
df_toronto[df_toronto['Neighborhood']=='Not assigned']

,PostalCode,Borough,Neighborhood


In [10]:
#(Unecessary, for practice reason) 2.assign neighborhood name with borough name if neighborhood not named

Borough=df_toronto['Borough']
Neighborhood=df_toronto['Neighborhood']

for borough, neighborhood in zip(Borough, Neighborhood):
    if neighborhood=='Not assigned':
        df_toronto['Neighborhood'].replace(to_replace='Not assigned', value=borough)

df_toronto

,PostalCode,Borough,Neighborhood
0,M3A,North York,Parkwoods
1,M4A,North York,Victoria Village
2,M5A,Downtown Toronto,"Regent Park, Harbourfront"
3,M6A,North York,"Lawrence Manor, Lawrence Heights"
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government"
...,...,...,...
98,M8X,Etobicoke,"The Kingsway, Montgomery Road, Old Mill North"
99,M4Y,Downtown Toronto,Church and Wellesley
100,M7Y,East Toronto,"Business reply mail Processing Centre, South C..."
101,M8Y,Etobicoke,"Old Mill South, King's Mill Park, Sunnylea, Hu..."


In [11]:
df_toronto.shape

(103, 3)

---




### Step 2. Get and load geo data




In [12]:
#Get geo datafram

geo_file='http://cocl.us/Geospatial_data'
df_geo=pd.read_csv(geo_file)

df_geo.head()

,Postal Code,Latitude,Longitude
0,M1B,43.806686,-79.194353
1,M1C,43.784535,-79.160497
2,M1E,43.763573,-79.188711
3,M1G,43.770992,-79.216917
4,M1H,43.773136,-79.239476


In [13]:
#Get geo dataframe ready to merge

df_geo.rename(columns={'Postal Code':'PostalCode'}, inplace=True)

In [14]:
#Join two dataframes

df_merged=df_toronto.join(df_geo.set_index('PostalCode'), on='PostalCode')

df_merged.head(10)

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M3A,North York,Parkwoods,43.753259,-79.329656
1,M4A,North York,Victoria Village,43.725882,-79.315572
2,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
3,M6A,North York,"Lawrence Manor, Lawrence Heights",43.718518,-79.464763
4,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
5,M9A,Etobicoke,"Islington Avenue, Humber Valley Village",43.667856,-79.532242
6,M1B,Scarborough,"Malvern, Rouge",43.806686,-79.194353
7,M3B,North York,Don Mills,43.745906,-79.352188
8,M4B,East York,"Parkview Hill, Woodbine Gardens",43.706397,-79.309937
9,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937


---




### Step 3. Cluster Neighborhood

In [15]:
#Pick out boroughs named with "Toronto", and work with them only

df_merged['Borough'].value_counts()

North York          24
Downtown Toronto    19
Scarborough         17
Etobicoke           12
Central Toronto      9
West Toronto         6
East Toronto         5
East York            5
York                 5
Mississauga          1
Name: Borough, dtype: int64

In [16]:
toronto_borough=['Downtown Toronto','Central Toronto','West Toronto','East Toronto']

df_tboro=df_merged[df_merged['Borough'].isin(toronto_borough)].reset_index(drop=True)

df_tboro.head()

,PostalCode,Borough,Neighborhood,Latitude,Longitude
0,M5A,Downtown Toronto,"Regent Park, Harbourfront",43.654260,-79.360636
1,M7A,Downtown Toronto,"Queen's Park, Ontario Provincial Government",43.662301,-79.389494
2,M5B,Downtown Toronto,"Garden District, Ryerson",43.657162,-79.378937
3,M5C,Downtown Toronto,St. James Town,43.651494,-79.375418
4,M4E,East Toronto,The Beaches,43.676357,-79.293031


See a toronto map before preceed forward

In [17]:
#Get folium
!conda install -c conda-forge folium=0.5.0 --yes
import folium

Solving environment: \ 
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::certifi-2020.6.20-py38_0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::certifi-2020.6.20-py38_0
  - anaconda/osx-64::certifi-2020.6.20-py38_0, anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::ca-certificates-2020.6.24-0
  - anaconda/osx-64::openssl-1.1.1g-h1de35cc_0, defaults/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::certifi-2020.6.20-py38_0
  - anaconda/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - defaults/osx-64::ca-certificates-2020.6.24-0, defaults/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::ca-certificates-2020.6.24-0, anaconda/osx-64::certifi-2020.6.20-py38_0, defaults/osx-64::openssl-1.1.1g-h1de35cc_0
  - anaconda/osx-64::ca-

In [18]:
#Google searched Toronto's lat and long
toronto_coordinates=[43.6532, -79.3832]

In [19]:
#create a toronto map
toronto_map=folium.Map(location=toronto_coordinates, zoom_start=14)

#add neighborhoods to map

for lat, lng, label in zip(df_tboro['Latitude'], df_tboro['Longitude'], df_tboro['Neighborhood']):
    folium.CircleMarker(
        [lat, lng], 
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(toronto_map)

toronto_map

In [20]:
#Unfortunately jupyter notebook won't display folium map with too many markers
map1=folium.Map(location=toronto_coordinates, zoom_start=14)

map1

In [21]:
#Get ready for Foursquare API

C_ID='J3CYFPKQMZFECQHPED1WLLHO3SM0YSXWIAMD0KV2UYVUPIEB'
C_SECRET='HXLYUWEBOGMZK5CQGIIBF3IO21K3X1GEQQYRKR2LL0KQFYN0'
VERSION='20191127'

print('My Foursquare client id is %s, client secret is %s, using version %s' % (C_ID, C_SECRET, VERSION))

My Foursquare client id is J3CYFPKQMZFECQHPED1WLLHO3SM0YSXWIAMD0KV2UYVUPIEB, client secret is HXLYUWEBOGMZK5CQGIIBF3IO21K3X1GEQQYRKR2LL0KQFYN0, using version 20191127


In [22]:
#def function of getting nearby parks
def getparks(names, latitudes, longitudes, radius=500):
    
    parks_list=[]
    search_parks='park'
    limit=30
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            C_ID, 
            C_SECRET, 
            VERSION, 
            lat, 
            lng,
            search_parks,
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['venues']
        
        # return only relevant information for each nearby venue
        parks_list.append([(name, lat, lng, v['name'], v['location']['lat'], v['location']['lng'], v['categories']) for v in results])

    nearby_parks = pd.DataFrame([item for parks_list in parks_list for item in parks_list])
    nearby_parks.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_parks)

In [25]:
#def function of getting nearby coffee shops

def getcoffee(names, latitudes, longitudes, radius=500):
    
    coffee_list=[]
    search_coffee='coffee'
    limit=30
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
        
        url='https://api.foursquare.com/v2/venues/search?&client_id={}&client_secret={}&v={}&ll={},{}&query={}&radius={}&limit={}'.format(
            C_ID,
            C_SECRET,
            VERSION,
            lat,
            lng,
            search_coffee,
            radius,
            limit)
        
        results=requests.get(url).json()['response']['venues']
        
        coffee_list.append([(
            name,
            lat,
            lng,
            v['name'],
            v['location']['lat'],
            v['location']['lng'],
            v['categories']) for v in results])
    
    nearby_coffee=pd.DataFrame([item for coffee_list in coffee_list for item in coffee_list])
    nearby_coffee.columns=['Neighborhood',
                           'Neighborhood Latitude',
                           'Neighborhood Longitude',
                           'Venue',
                           'Venue Latitude',
                           'Venue Longitude',
                           'Venue Category']
    
    return(nearby_coffee)

In [26]:
tboro_parks=getparks(names=df_tboro['Neighborhood'], latitudes=df_tboro['Latitude'], longitudes=df_tboro['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [27]:
print(tboro_parks.shape)
tboro_parks.head()

(603, 7)


,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,"Regent Park, Harbourfront",43.65426,-79.360636,Orphan's Greenspace Dog Park,43.654654,-79.362501,"[{'id': '4bf58dd8d48988d1e5941735', 'name': 'D..."
1,"Regent Park, Harbourfront",43.65426,-79.360636,Wilfrid Park,43.650999,-79.364915,"[{'id': '4bf58dd8d48988d124941735', 'name': 'O..."
2,"Regent Park, Harbourfront",43.65426,-79.360636,Parliament Square Park,43.650264,-79.362195,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P..."
3,"Regent Park, Harbourfront",43.65426,-79.360636,Power Park Off- leash,43.654763,-79.362408,"[{'id': '4bf58dd8d48988d1e5941735', 'name': 'D..."
4,"Regent Park, Harbourfront",43.65426,-79.360636,Percy Park,43.655180,-79.357421,"[{'id': '4bf58dd8d48988d163941735', 'name': 'P..."


In [30]:
tboro_coffee=getcoffee(names=df_tboro['Neighborhood'], latitudes=df_tboro['Latitude'], longitudes=df_tboro['Longitude'])

Regent Park, Harbourfront
Queen's Park, Ontario Provincial Government
Garden District, Ryerson
St. James Town
The Beaches
Berczy Park
Central Bay Street
Christie
Richmond, Adelaide, King
Dufferin, Dovercourt Village
Harbourfront East, Union Station, Toronto Islands
Little Portugal, Trinity
The Danforth West, Riverdale
Toronto Dominion Centre, Design Exchange
Brockton, Parkdale Village, Exhibition Place
India Bazaar, The Beaches West
Commerce Court, Victoria Hotel
Studio District
Lawrence Park
Roselawn
Davisville North
Forest Hill North & West, Forest Hill Road Park
High Park, The Junction South
North Toronto West, Lawrence Park
The Annex, North Midtown, Yorkville
Parkdale, Roncesvalles
Davisville
University of Toronto, Harbord
Runnymede, Swansea
Moore Park, Summerhill East
Kensington Market, Chinatown, Grange Park
Summerhill West, Rathnelly, South Hill, Forest Hill SE, Deer Park
CN Tower, King and Spadina, Railway Lands, Harbourfront West, Bathurst Quay, South Niagara, Island airport
R

In [ ]:
print(tboro_coffee.shape)
tboro_coffee.head()

In [ ]:
# function that extracts the category of the venue
def get_category_type(row):
    try:
        categories_list = row['Venue Category']
    except:
        categories_list = row['venue.categories']
        
    if len(categories_list) == 0:
        return None
    else:
        return categories_list[0]['name']

# filter the category for each row
tboro_parks['Venue Category'] = tboro_parks.apply(get_category_type, axis=1)
tboro_coffee['Venue Category']=tboro_coffee.apply(get_category_type, axis=1)


In [ ]:
tboro_parks.head()

In [ ]:
tboro_coffee.head()

In [ ]:
#keep only park or coffee shop in venue category

tboro_parks=tboro_parks[tboro_parks['Venue Category']=='Park'].reset_index(drop=True)
tboro_coffee=tboro_coffee[tboro_coffee['Venue Category']=='Coffee Shop'].reset_index(drop=True)

In [ ]:
#count number of parks or coffee shops in neighborhood
park_counts = tboro_parks['Neighborhood'].value_counts().rename_axis('Neighborhood').to_frame('Park Counts').reset_index()
coffee_counts = tboro_coffee['Neighborhood'].value_counts().rename_axis('Neighborhood').to_frame('Coffee Counts').reset_index()

In [ ]:
print(park_counts.shape)
park_counts.head()

In [ ]:
print(coffee_counts.shape)
coffee_counts.head()

In [ ]:
#add park and coffee counts to df_tboro

df_tboro=df_tboro.join(park_counts.set_index('Neighborhood'), on='Neighborhood')
df_tboro=df_tboro.join(coffee_counts.set_index('Neighborhood'), on='Neighborhood')


In [ ]:
df_tboro

In [ ]:
#assign 0 to Nan
df_tboro.fillna(0, inplace=True)

#check the df again
df_tboro

In [ ]:
#Now, ready to cluster neighborhoods based on number of coffee shops and parks, and their location

from sklearn.cluster import KMeans
import matplotlib.cm as cm
import matplotlib.colors as colors

In [ ]:
tboro_cluster=df_tboro.loc[:,['Latitude', 'Longitude', 'Park Counts','Coffee Counts']]
tboro_cluster.head()

In [ ]:
km = KMeans(n_clusters=5, random_state=0)
km.fit(tboro_cluster)
km.labels_[0:10]

In [ ]:
#label neighborhoods with cluster labels
df_tboro.insert(0, 'Cluster Labels', km.labels_)
df_tboro.head()

In [ ]:
#visualize clusters
tboro_clustered = folium.Map(location=toronto_coordinates, zoom_start=12)

# set color scheme for the clusters
x = np.arange(5)
ys = [i + x + (i*x)**2 for i in range(5)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(df_tboro['Latitude'], df_tboro['Longitude'], df_tboro['Neighborhood'], df_tboro['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(tboro_clustered)
       
tboro_clustered